In [1]:
import numpy as np
import pandas as pd
from lmfit import Model

**Hydrogenation of Ethylene to Ethane**

You can download Fogler's book from here: https://doku.pub/download/h-scott-fogler-essentials-of-chemical-reaction-engineering-z0x2ze484wqn

$H_2 + C_2H_4 -> C_2H_6$

Carry out a nonlinear regression analysis on the data given and determine which rate law best describes the **differential reactor (DR)** data.

$(a) -r'_E = \frac{kP_EP_H}{1+K_{EA}P_{EA}+K_EP_E}$

$(b) -r'_E = \frac{kP_EP_H}{1+K_{E}P_{E}}$

$(c) -r'_E = \frac{kP_EP_H}{(1+K_{E}P_{E})^2}$

$(d) -r'_E = kP_E^aP_H^b $

In [2]:
df = pd.DataFrame()
df['Run Number'] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
df['Reaction Rate (mol/kg$_{cat}$/s)'] = [1.04, 3.13, 5.21, 3.82, 4.19, 2.391, 3.867, 2.199, 0.75]
df['P$_E$ (atm)'] = [1., 1., 1., 3., 5., 0.5, 0.5, 0.5, 0.5]
df['P$_{EA}$ (atm)'] = [1., 1., 1., 1., 1., 1., 0.5, 3., 5.]
df['P$_H$ (atm)'] = [1., 3., 5., 3., 3., 3., 5., 3., 1.]

df = df.set_index('Run Number')

df

,Reaction Rate (mol/kg$_{cat}$/s),P$_E$ (atm),P$_{EA}$ (atm),P$_H$ (atm)
Run Number,,,,
1,1.040,1.0,1.0,1.0
2,3.130,1.0,1.0,3.0
3,5.210,1.0,1.0,5.0
4,3.820,3.0,1.0,3.0
5,4.190,5.0,1.0,3.0
6,2.391,0.5,1.0,3.0
7,3.867,0.5,0.5,5.0
8,2.199,0.5,3.0,3.0
9,0.750,0.5,5.0,1.0


In [3]:
ydata = df.iloc[:, 0].T.values
xdata = df.iloc[:, 1:].T.values

### model (a)

In [4]:
def y_calc_A(x, k, K_EA, K_E):
    
    num = k*x[0]*x[2]
    denom = 1+K_EA*x[1]+K_E*x[0]
    y = num/denom
    
    return y

model_A = Model(y_calc_A)

result_A = model_A.fit(ydata, x=xdata, k=1.0, K_EA=1.0, K_E=1.0)

print(result_A.fit_report())

[[Model]]
    Model(y_calc_A)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 38
    # data points      = 9
    # variables        = 3
    chi-square         = 0.02961672
    reduced chi-square = 0.00493612
    Akaike info crit   = -45.4497667
    Bayesian info crit = -44.8580930
[[Variables]]
    k:     3.34787630 +/- 0.15980989 (4.77%) (init = 1)
    K_EA:  0.04284145 +/- 0.02916203 (68.07%) (init = 1)
    K_E:   2.21107624 +/- 0.13031663 (5.89%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(k, K_E)    = 0.977
    C(k, K_EA)   = 0.689
    C(K_EA, K_E) = 0.581


### model (b)

In [5]:
def y_calc_B(x, k, K_E):
    
    num = k*x[0]*x[2]
    denom = 1+K_E*x[0]
    y = num/denom
    
    return y

model_B = Model(y_calc_B)

result_B = model_B.fit(ydata, x=xdata, k=1.0, K_E=1.0)

print(result_B.fit_report())

[[Model]]
    Model(y_calc_B)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 16
    # data points      = 9
    # variables        = 2
    chi-square         = 0.04237352
    reduced chi-square = 0.00605336
    Akaike info crit   = -44.2261049
    Bayesian info crit = -43.8316558
[[Variables]]
    k:    3.18678195 +/- 0.12178901 (3.82%) (init = 1)
    K_E:  2.10133338 +/- 0.11159830 (5.31%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(k, K_E) = 0.978


### model (c)

In [6]:
def y_calc_C(x, k, K_E):
    
    num = k*x[0]*x[2]
    denom = (1+K_E*x[0])**2
    y = num/denom
    
    return y

model_C = Model(y_calc_C)

result_C = model_C.fit(ydata, x=xdata, k=1.0, K_E=1.0)

print(result_C.fit_report())

[[Model]]
    Model(y_calc_C)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 30
    # data points      = 9
    # variables        = 2
    chi-square         = 0.43612186
    reduced chi-square = 0.06230312
    Akaike info crit   = -23.2435235
    Bayesian info crit = -22.8490743
[[Variables]]
    k:    2.00878307 +/- 0.11255632 (5.60%) (init = 1)
    K_E:  0.36166703 +/- 0.02634703 (7.28%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(k, K_E) = 0.890


### model (d)

In [7]:
def y_calc_D(x, k, a, b):
    
    return k*x[0]**a*x[2]**b

model_D = Model(y_calc_D)

result_D = model_D.fit(ydata, x=xdata, k =1.0, a=1.0, b=1.0)

print(result_D.fit_report())

[[Model]]
    Model(y_calc_D)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 21
    # data points      = 9
    # variables        = 3
    chi-square         = 0.29722302
    reduced chi-square = 0.04953717
    Akaike info crit   = -24.6944739
    Bayesian info crit = -24.1028001
[[Variables]]
    k:  0.89402691 +/- 0.10498880 (11.74%) (init = 1)
    a:  0.25844116 +/- 0.02897048 (11.21%) (init = 1)
    b:  1.06155141 +/- 0.08553385 (8.06%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(k, b) = -0.979
    C(k, a) = -0.487
    C(a, b) = 0.442


### Observations

*We now examine the reduced chi-square and range of variables themselves. 
The reduced chi-square is reasonable and in fact the smallest of all the models at 0.0049.*

*The 95% confidence limit means that if the experiment were 
run 100 times and then 95 times it would fall within the specified range (e.g., 0.04284145 +/- 0.02916203 for model (a).*

*If the lower limit within the confidence interval leads to a negative value then the model should be rejected.*